In [1]:
import numpy as np
import pandas as pd
import requests
import json

In [2]:
url= "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2"

In [3]:
#Find pair id for UNI/WETH pair
query = """
 {pairs (where :{token0 : "0x1f9840a85d5af5bf1d1762f925bdaddc4201f984", token1 :"0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"}) { 
    id
    createdAtTimestamp
  }
}
"""
r = requests.post(url, json={'query':query})
json_data = json.loads(r.text)
tid = json_data['data']['pairs'][0]['id']

In [23]:
query="""{
swaps(orderBy: timestamp, orderDirection: desc, where:
 { pair: "0xd3d2e2692501a5c9ca623199d38826e513033a17" }
) {
     timestamp
     amount0In
     amount0Out
     amount1In
     amount1Out
     amountUSD
 }
}"""
r = requests.post(url, json={'query': query})

In [24]:
json_data = json.loads(r.text)
df = pd.DataFrame(json_data["data"]["swaps"])

In [25]:
side =[]
for i in range(0,max(df.index)+1):
    if(df.iloc[i]["amount0In"]=='0'):
        side.append("Sell")
    else :
        side.append("Buy")
df["Side"]=side
df["Base Currency Qty Traded"] = df["amount0In"]+df["amount0Out"]
df["Quote currency qty traded"] = df["amount1In"]+df["amount1Out"]
df=df.drop(columns = ["amount0In","amount0Out","amount1In","amount1Out"])

In [28]:
df

,amountUSD,timestamp,Side,Base Currency Qty Traded,Quote currency qty traded
0,165.2923852675834444879762419932073,1624381209,Buy,100,00.086564790527989745
1,763.1730343124259404677263094223611,1624381083,Buy,46.2215758817195130880,00.400125777030770893
2,283.4921945543797369136557054695254,1624381048,Buy,17.1674217103251903690,00.148617063768743025
3,7300.577459008628368116317728538263,1624381033,Sell,0439.536348087668651315,3.8272682854588376030
4,494.702831912350293246700011715996,1624381024,Buy,29.9767079467853842240,00.259413062270487088
...,...,...,...,...,...
95,114502.619374814223322959870765263,1624366914,Sell,07834.425009086785183845,63.5873836383727108350
96,24768.02346405682772236857495925829,1624366739,Buy,1707.4470370643130449920,013.740606365793356435
97,6694.864744898964108933787900780879,1624366694,Buy,463.8482000144627748420,03.736123745544059242
98,193642.5071360116066759611488382967,1624366628,Sell,013434.392963018516868521,108.2818472594930917890
